In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
%matplotlib inline

# Import API key
from configG import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
path_file = os.path.join('../Weather_Database', 'WeatherPy_Database.csv')
city_data_df = pd.read_csv(path_file)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,63.32,85,100,20.00,overcast clouds
1,1,Cairns,AU,-16.9167,145.7667,83.10,78,40,3.44,scattered clouds
2,2,Bijar,IR,35.8668,47.6051,27.91,94,82,2.48,broken clouds
3,3,Satipo,PE,-11.2522,-74.6386,73.31,89,100,0.76,moderate rain
4,4,Butaritari,KI,3.0707,172.7902,81.72,79,29,17.45,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input("For a more pleasant trip, what is the minimum temperature you expect? "))
maximum_temp = float(input("For a more pleasant trip, what is the maximum temperature you expect? "))

For a more pleasant trip, what is the minimum temperature you expect? 40
For a more pleasant trip, what is the maximum temperature you expect? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
favorite_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= minimum_temp) & (city_data_df["Max Temp"] <= maximum_temp)]
favorite_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,63.32,85,100,20.00,overcast clouds
3,3,Satipo,PE,-11.2522,-74.6386,73.31,89,100,0.76,moderate rain
5,5,Rikitea,PF,-23.1203,-134.9692,79.47,71,58,17.63,broken clouds
6,6,Yongzhou,CN,26.4559,111.5988,40.39,95,100,3.31,overcast clouds
8,8,Itoman,JP,26.1247,127.6694,60.82,72,75,5.75,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(favorite_cities_df))
favorite_cities_df.count()

379


City_ID                379
City                   379
Country                373
Lat                    379
Lng                    379
Max Temp               379
Humidity               379
Cloudiness             379
Wind Speed             379
Current Description    379
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = favorite_cities_df.dropna()
print(clean_df.count())


City_ID                373
City                   373
Country                373
Lat                    373
Lng                    373
Max Temp               373
Humidity               373
Cloudiness             373
Wind Speed             373
Current Description    373
dtype: int64


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,63.32,overcast clouds,22.2855,114.1577,
3,Satipo,PE,73.31,moderate rain,-11.2522,-74.6386,
5,Rikitea,PF,79.47,broken clouds,-23.1203,-134.9692,
6,Yongzhou,CN,40.39,overcast clouds,26.4559,111.5988,
8,Itoman,JP,60.82,broken clouds,26.1247,127.6694,
13,Arraial Do Cabo,BR,71.46,overcast clouds,-22.9661,-42.0278,
14,Cape Town,ZA,65.26,clear sky,-33.9258,18.4232,
16,Ancud,CL,60.33,few clouds,-41.8697,-73.8203,
17,Huarmey,PE,72.99,overcast clouds,-10.0681,-78.1522,
18,Formosa Do Rio Preto,BR,75.49,overcast clouds,-11.0483,-45.1931,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
       # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,63.32,overcast clouds,22.2855,114.1577,Mini Hotel Central
3,Satipo,PE,73.31,moderate rain,-11.2522,-74.6386,Bungalows El Palmiche
5,Rikitea,PF,79.47,broken clouds,-23.1203,-134.9692,People ThankYou
6,Yongzhou,CN,40.39,overcast clouds,26.4559,111.5988,Jialong Hotel
8,Itoman,JP,60.82,broken clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))